# Student-course-grade example in SQL

*Following cell loads the SQL extension for jupyter notebooks and sets up an empty SQLite3 database in memory. Ater executing it, you should see ``'Connected: @None'``. If you get an error instead, you have probably forgotten to install ipython-sql. In that case, open a terminal (run ``cmd`` on Windows) and run: ``pip install ipython-sql``*

In [1]:
%load_ext sql
%sql sqlite://
%sql PRAGMA foreign_keys = ON;

 * sqlite://
Done.


[]

Now, we are able to execute SQL statements to create tables for students, courses and grades (note that ``%%sql`` at the beginning is not a part of SQL, but an information for the kernel (a program which executes the code in cells) that the cell contains SQL and not a Python code):

In [2]:
%%sql
CREATE TABLE student(
  id int NOT NULL PRIMARY KEY,
  first_name varchar(255) NOT NULL,
  last_name varchar(255) NOT NULL
);
CREATE TABLE course(
  id int NOT NULL PRIMARY KEY,
  code varchar(30) NOT NULL UNIQUE,
  title varchar(255)
);
CREATE TABLE grade(
  student_id int NOT NULL,
  course_id int NOT NULL,
  grade char(1),
  PRIMARY KEY(student_id, course_id),
  FOREIGN KEY(student_id) REFERENCES student(id),
  FOREIGN KEY(course_id) REFERENCES course(id)
);

 * sqlite://
Done.
Done.
Done.


[]

Note that we have used ``NOT NULL`` for each of the attributes except for the course title and the grade.

Now, let’s insert some data:

In [3]:
%%sql
INSERT INTO student(id, first_name, last_name) VALUES (1, 'Homer', 'Simpson');
INSERT INTO student(id, first_name, last_name) VALUES (2, 'Marge', 'Bouvier');

 * sqlite://
1 rows affected.
1 rows affected.


[]

We can look at the contents of the table by running the following SQL query:

In [4]:
%%sql
SELECT * FROM student

 * sqlite://
Done.


id,first_name,last_name
1,Homer,Simpson
2,Marge,Bouvier


What happens if we now try to insert another tuple in the table with ID 2 (note that there already exists a student with that ID)?

*N.B.: Only ``INSERT INTO ...`` is an SQL, the rest is a Python code catching and priting an error from the database. We will use this later too.*

In [5]:
try:
    %sql INSERT INTO student(id, first_name, last_name) VALUES (2, 'Bart', 'Simpson');
except Exception as e:
    print(e)

 * sqlite://
(sqlite3.IntegrityError) UNIQUE constraint failed: student.id [SQL: "INSERT INTO student(id, first_name, last_name) VALUES (2, 'Bart', 'Simpson');"] (Background on this error at: http://sqlalche.me/e/gkpj)


We get an error: ``UNIQUE constraint failed: student.id``.

Let’s now add two courses to our database: 1DL301 Database Design I and 1DL321 for which we do not know the name (recall that we have allowed NULLs for the course title):

In [6]:
%%sql
INSERT INTO course(id, code, title) VALUES (1, '1DL301', 'Database Design I');
INSERT INTO course(id, code) VALUES (2, '1DL321');

 * sqlite://
1 rows affected.
1 rows affected.


[]

The title for 1DL321 is "Compiler Design I" so let's update the course:

In [7]:
%%sql
UPDATE course SET title='Compiler Design I' WHERE id=2

 * sqlite://
1 rows affected.


[]

To see the list of all courses, we can run the following query:

In [8]:
%%sql
SELECT * FROM course

 * sqlite://
Done.


id,code,title
1,1DL301,Database Design I
2,1DL321,Compiler Design I


If we at this point try to insert a grade (say, U) for Homer Simpson (i.e., the student with ID 1) and the course with ID 3 by running:

In [9]:
try:
    %sql INSERT INTO grade(student_id, course_id, grade) VALUES (1, 3, 'U')
except Exception as e:
    print(e)

 * sqlite://
(sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "INSERT INTO grade(student_id, course_id, grade) VALUES (1, 3, 'U')"] (Background on this error at: http://sqlalche.me/e/gkpj)


we will get an error (``FOREIGN KEY constraint failed``) since we are trying to refer to a non-existing course.

Alright, the grade is actually for the Database Design I course (with ID 1). Marge took that course too but with the grade of 5:

In [10]:
%%sql
INSERT INTO grade(student_id, course_id, grade) VALUES (1, 1, 'U');
INSERT INTO grade(student_id, course_id, grade) VALUES (2, 1, '5')

 * sqlite://
1 rows affected.
1 rows affected.


[]

Homer was not doing great at UU (he has not finished any other course) and returned to Springfield. We want to remove him from our database:

In [11]:
try:
    %sql DELETE FROM student WHERE id=1
except Exception as e:
    print(e)

 * sqlite://
(sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: 'DELETE FROM student WHERE id=1'] (Background on this error at: http://sqlalche.me/e/gkpj)


This does not work since there are rows in other tables which are referencing Homer (there is one grade for him in the grade table). We could remove these first and then remove Homer, but let's try something else instead.

Let's drop the grade table (the content will be removed too!) and re-create it again with ``ON DELETE CASCADE`` for both foreign keys:

In [12]:
%%sql
-- Drop the table:
DROP TABLE grade;
-- Create the table again:
CREATE TABLE grade(
  student_id int NOT NULL,
  course_id int NOT NULL,
  grade char(1),
  PRIMARY KEY(student_id, course_id),
  FOREIGN KEY(student_id) REFERENCES student(id) ON DELETE CASCADE,
  FOREIGN KEY(course_id) REFERENCES course(id) ON DELETE CASCADE
);
-- Insert the original content into the table:
INSERT INTO grade(student_id, course_id, grade) VALUES (1, 1, 'U');
INSERT INTO grade(student_id, course_id, grade) VALUES (2, 1, '5')

 * sqlite://
Done.
Done.
Done.
1 rows affected.


[]

If we now try to delete Homer from the student table:

In [13]:
%%sql
DELETE FROM student WHERE id=1

 * sqlite://
1 rows affected.


[]

No error. What is the content of the student table? And what is the content of the grade table?

In [14]:
%%sql
SELECT * FROM student;

 * sqlite://
Done.


id,first_name,last_name
2,Marge,Bouvier


In [15]:
%%sql
SELECT * FROM grade;

 * sqlite://
Done.


student_id,course_id,grade
2,1,5


Homer was indeed removed from the student table so were all the tuples from the grade table referencing the removed student.

The ``ON DELETE CASCADE`` specification says that all grade rows for a particular student are going to be removed when that student is removed. (And similarly, all grade rows for a particular course are going to be removed when that course is removed because of the second ``ON DELETE CASCADE``.)

Very similar situation occurs if we try to change the primary key for a student. If we want to change the ID of Marge Simpson to, say, 100 (nobody has this ID so we are allowed to do that):

In [16]:
try:
    %sql UPDATE student SET id=100 WHERE id=2
except Exception as e:
    print(e)

 * sqlite://
(sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: 'UPDATE student SET id=100 WHERE id=2'] (Background on this error at: http://sqlalche.me/e/gkpj)


We will get an error as well (Error: ``FOREIGN KEY constraint failed``). The reason is that the grade table has a row saying "student with ID 2" got grade of 5 in "course with ID 1" and after changing the ID there would be no student with ID 1. We can force updating student IDs in other tables by adding ``ON UPDATE CASCADE``:

In [17]:
%%sql
-- Drop the table:
DROP TABLE grade;
-- Create the table again:
CREATE TABLE grade(
  student_id int NOT NULL,
  course_id int NOT NULL,
  grade char(1),
  PRIMARY KEY(student_id, course_id),
  FOREIGN KEY(student_id) REFERENCES student(id) ON DELETE CASCADE ON UPDATE CASCADE,
  FOREIGN KEY(course_id) REFERENCES course(id) ON DELETE CASCADE ON UPDATE CASCADE
);
-- Insert the original content into the table:
INSERT INTO grade(student_id, course_id, grade) VALUES (2, 1, '5');

 * sqlite://
Done.
Done.
Done.


[]

Now let's try again:

In [18]:
%%sql
UPDATE student SET id=100 WHERE id=2

 * sqlite://
1 rows affected.


[]

The student table looks like expected (Marge has the ID 100):

In [19]:
%%sql
SELECT * FROM student

 * sqlite://
Done.


id,first_name,last_name
100,Marge,Bouvier


What about the grade table?

In [20]:
%%sql
SELECT * FROM grade

 * sqlite://
Done.


student_id,course_id,grade
100,1,5


The ``student_id``s equal to 2 were updated to 100.

Some final notes:

* The keywords in SQL are case-insensitive (i. e. you can write ``SELECT * FROM grade`` or ``select * from grade``).
* The names of tables and columns might be case-sensitive or case-insensitive – depends on the RDBM you use.
* Several SQLs are separated by semicolons.